# This is a notebook for downloading images from the Azure Bing Image Search API

In [ ]:
# python modules
import os
from pathlib import Path

# python image handling module
from PIL import Image

# azure modules
from azure.cognitiveservices.search.imagesearch import ImageSearchClient as api
from msrest.authentication import CognitiveServicesCredentials as auth

# fastai modules
from fastcore.foundation import L
from fastai.data.external import download_url
from fastai.data.transforms import get_image_files
from fastai.vision.utils import download_images, verify_images 

In [ ]:
# function for searching for images via azure bing image search api
# returns URLs
def search_images_bing(key, term, min_sz=128):
    client = api('https://api.cognitive.microsoft.com', auth(key))
    return L(client.images.search(query=term, count=150, min_height=min_sz, min_width=min_sz).value)

In [ ]:
# store api key in variable
key = os.environ.get('AZURE_SEARCH_KEY', 'XXX') # replace 'XXX' with actual key

In [ ]:
# check if search_images_bing function works
results = search_images_bing(key, 'brushtail')
ims = results.attrgot('content_url')
len(ims)

In [ ]:
# test download function
dest = 'images/brushtail.jpg'
download_url(ims[0], dest)

In [ ]:
# show image
im = Image.open(dest)
MAX_SIZE = (128, 128) 
im.thumbnail(MAX_SIZE) 
im

In [ ]:
# specify search and set path
marsupial_types = ('brushtail','opossum')
path = Path('marsupials')

In [ ]:
# iterate over marsupial types and store downloaded images in respective directories
if not path.exists():
    path.mkdir()
    for o in marsupial_types:
        print(o)
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, o)
        download_images(dest, urls=results.attrgot('content_url'))

In [ ]:
# check the image files
fns = get_image_files(path)
fns

In [ ]:
# check if downloaded files are actually images and display how many files are not images
failed = verify_images(fns)
failed

In [ ]:
# delete non-image files
failed.map(Path.unlink);